## Unzip files, create 16 formatted dataframes, first filling dataframes with daily data

In [1]:
import zipfile
import os
from pathlib import Path
import pprint
import pandas as pd

In [2]:
def unzip_given_file_to_folder(file_path, folder):
    with zipfile.ZipFile(file_path, 'r') as z:
        z.extractall(folder)

In [3]:
unzipped_data = '/media/lenalexu/64522E74522E4ADE/D/Whitireia/ARP/data/unzipped_data/'

In [4]:
os.chdir('/media/lenalexu/64522E74522E4ADE/D/Whitireia/ARP/data/zipped_data')
for file in os.listdir('/media/lenalexu/64522E74522E4ADE/D/Whitireia/ARP/data/zipped_data'):
    if file.endswith('.zip'):
        file_path = os.path.abspath(file)
        unzip_given_file_to_folder(file_path=file_path, folder=unzipped_data)
    else:
        print(f"{file} - Not a ZIP file!")

In [5]:
list_csv_files = os.listdir(unzipped_data)
print(len(list_csv_files))
list_csv_files

229


['41559__Temperature__D.csv',
 '41559__Temperature__H.csv',
 '41559__Temperature__unknown.csv',
 '41559__Wind__all_data.csv',
 '41559__Wind__D.csv',
 '41559__Wind__H.csv',
 '8567__Rain__all_data.csv',
 '8567__Rain__D.csv',
 '8567__Rain__M.csv',
 '8567__Rain__S.csv',
 '8567__Screen_Observations__A.csv',
 '8567__Screen_Observations__D.csv',
 '8567__Screen_Observations__M.csv',
 '8567__Screen_Observations__S.csv',
 '8567__Temperature__A.csv',
 '8567__Temperature__all_data.csv',
 '8567__Temperature__D.csv',
 '8567__Temperature__unknown.csv',
 '8567__Wind__all_data.csv',
 '8567__Wind__D.csv',
 '8567__Wind__M.csv',
 '8567__Wind__S.csv',
 '111__RainD.csv',
 '111__Screen_ObservationsD.csv',
 '111__TemperatureD.csv',
 '111__WindD.csv',
 '12442__Rain__all_data.csv',
 '12442__Rain__D.csv',
 '12442__Rain__H.csv',
 '12442__Rain__Q.csv',
 '12442__Rain__R.csv',
 '12442__Screen_Observations__H.csv',
 '12442__Temperature__all_data.csv',
 '12442__Temperature__D.csv',
 '12442__Temperature__H.csv',
 '1244

In [6]:
df_stations = pd.read_csv('/media/lenalexu/64522E74522E4ADE/D/Whitireia/ARP/clustering-wellington-weather-stations/stations.csv')

In [7]:
columns = ['station_id', 'Observation time UTC', 'Temperature', 'Screen_Observations', 'Rain', 'Wind']
dict_stations = {}
df = pd.DataFrame(columns=columns)
df['Observation time UTC'] = pd.to_datetime(df['Observation time UTC']).dt.date
dict_stations = {station_id: df for station_id in df_stations['StationID']}
print(sorted(dict_stations))

[2592, 2685, 3145, 3445, 8567, 12442, 18234, 21938, 25354, 25531, 31857, 40750, 40984, 41212, 41229, 41559]


In [8]:
def get_column_df_from_csv(csv_file_path: Path, column: str) -> pd.DataFrame | None: # temp df
    column_df = pd.read_csv(csv_file_path)
    column_df['Observation time UTC'] = pd.to_datetime(column_df['Observation time UTC']).dt.date
    if column == 'Temperature' or column == 'Rain':
        column_df.drop(column_df[column_df['Frequency [D/H/S]'] != 'Daily'].index, inplace=True)
    elif column == 'Wind':
        column_df.drop(column_df[column_df['Frequency [D/H]'] != 'Daily'].index, inplace=True)  
    
    return column_df

In [9]:
def merge_station_df_with_column_df(station_df: pd.DataFrame, column_df: pd.DataFrame, column: str) -> pd.DataFrame:
    column_to_columns = {
    'Temperature': 'Mean Temperature [Deg C]',
    'Rain': 'Rainfall [mm]',
    'Wind': 'Speed [m/s]',
    'Screen_Observations': 'Relative humidity [%]'
    }

    station_df = pd.merge(station_df, column_df[['Observation time UTC', column_to_columns[column]]], on='Observation time UTC', how='outer')
    station_df[column] = station_df[column_to_columns[column]]
    station_df = station_df[columns].sort_values('Observation time UTC', ascending=False)

    return station_df

### Filling climatic parameters of each station with daily data

In [10]:
for file_name in list_csv_files:
    try:
        column, param = file_name.split('__')[1:3]
    except:
        continue

    station_id: int = int(file_name.split('__')[0])

    if param.startswith('D.csv') and station_id in dict_stations:   # Daily data
        print(f"station_id: {station_id}, column: {column}, param:{param}")
       
        csv_file_path = Path(unzipped_data + file_name)
        column_df = get_column_df_from_csv(csv_file_path=csv_file_path, column=column)

        station_df = dict_stations[station_id]
        
        merged_station_df = merge_station_df_with_column_df(station_df=station_df, column_df=column_df, column=column)
        merged_station_df['station_id'] = merged_station_df['station_id'].fillna(station_id)
        dict_stations[station_id] = merged_station_df

display(dict_stations[40750])

station_id: 41559, column: Temperature, param:D.csv
station_id: 41559, column: Wind, param:D.csv
station_id: 8567, column: Rain, param:D.csv
station_id: 8567, column: Screen_Observations, param:D.csv
station_id: 8567, column: Temperature, param:D.csv
station_id: 8567, column: Wind, param:D.csv
station_id: 12442, column: Rain, param:D.csv
station_id: 12442, column: Temperature, param:D.csv
station_id: 18234, column: Rain, param:D.csv


/tmp/ipykernel_16944/402271887.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_station_df['station_id'] = merged_station_df['station_id'].fillna(station_id)
/tmp/ipykernel_16944/402271887.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_station_df['station_id'] = merged_station_df['station_id'].fillna(station_id)
/tmp/ipykernel_16944/3720988089.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.


station_id: 2685, column: Temperature, param:D.csv
station_id: 2685, column: Wind, param:D.csv
station_id: 3145, column: Rain, param:D.csv
station_id: 3145, column: Screen_Observations, param:D.csv
station_id: 3145, column: Temperature, param:D.csv


/tmp/ipykernel_16944/402271887.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_station_df['station_id'] = merged_station_df['station_id'].fillna(station_id)
/tmp/ipykernel_16944/402271887.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_station_df['station_id'] = merged_station_df['station_id'].fillna(station_id)


station_id: 3145, column: Wind, param:D.csv
station_id: 31857, column: Rain, param:D.csv
station_id: 31857, column: Screen_Observations, param:D.csv
station_id: 31857, column: Temperature, param:D.csv
station_id: 3445, column: Rain, param:D.csv
station_id: 3445, column: Screen_Observations, param:D.csv
station_id: 3445, column: Temperature, param:D.csv


...

station_id: 2685, column: Rain, param:D.csv
station_id: 2685, column: Screen_Observations, param:D.csv
station_id: 3445, column: Wind, param:D.csv
station_id: 40750, column: Rain, param:D.csv
station_id: 40750, column: Temperature, param:D.csv
station_id: 40984, column: Rain, param:D.csv
station_id: 40984, column: Temperature, param:D.csv
station_id: 40984, column: Wind, param:D.csv
station_id: 18234, column: Temperature, param:D.csv
station_id: 18234, column: Wind, param:D.csv
station_id: 21938, column: Rain, param:D.csv


/tmp/ipykernel_16944/402271887.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_station_df['station_id'] = merged_station_df['station_id'].fillna(station_id)
/tmp/ipykernel_16944/402271887.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_station_df['station_id'] = merged_station_df['station_id'].fillna(station_id)


station_id: 21938, column: Temperature, param:D.csv
station_id: 21938, column: Wind, param:D.csv
station_id: 25354, column: Rain, param:D.csv
station_id: 25354, column: Screen_Observations, param:D.csv
station_id: 25354, column: Temperature, param:D.csv
station_id: 41212, column: Rain, param:D.csv
station_id: 41212, column: Temperature, param:D.csv
station_id: 41212, column: Wind, param:D.csv
station_id: 41229, column: Rain, param:D.csv
station_id: 41229, column: Screen_Observations, param:D.csv
station_id: 41229, column: Temperature, param:D.csv
station_id: 41229, column: Wind, param:D.csv
station_id: 41559, column: Rain, param:D.csv


/tmp/ipykernel_16944/402271887.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_station_df['station_id'] = merged_station_df['station_id'].fillna(station_id)
/tmp/ipykernel_16944/402271887.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_station_df['station_id'] = merged_station_df['station_id'].fillna(station_id)


,station_id,Observation time UTC,Temperature,Screen_Observations,Rain,Wind
3629,40750,2024-12-11,NaN,NaN,NaN,3.20
3628,40750,2024-12-10,NaN,NaN,NaN,4.34
3627,40750,2024-12-09,NaN,NaN,NaN,4.38
3626,40750,2024-12-08,NaN,NaN,NaN,7.42
3625,40750,2024-12-07,NaN,NaN,0.0,3.48
...,...,...,...,...,...,...
4,40750,2015-01-07,15.8,NaN,0.0,3.61
3,40750,2015-01-06,19.7,NaN,0.6,3.61
2,40750,2015-01-05,19.0,NaN,0.0,4.22
1,40750,2015-01-04,16.0,NaN,0.0,2.99


In [36]:
for id in dict_stations:
    path_filled_stations = Path('/media/lenalexu/64522E74522E4ADE/D/Whitireia/ARP/data/stations_filled_daily/'+str(id)+'.csv')
    dict_stations[id].to_csv(path_filled_stations)

print(os.listdir('/media/lenalexu/64522E74522E4ADE/D/Whitireia/ARP/data/stations_filled_daily/'))

['12442.csv', '18234.csv', '21938.csv', '25354.csv', '25531.csv', '2592.csv', '2685.csv', '3145.csv', '31857.csv', '3445.csv', '40750.csv', '40984.csv', '41212.csv', '41229.csv', '41559.csv', '8567.csv']
